# Implementation of GraphSAGE

Vincent Wilmet \
rmbr to use GPU

In [1]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import torch
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 25.4 GB  | Proc size: 1.2 GB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total 16280MB


### init

In [3]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git
!pip install ogb
!pip install umap-learn

Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-_yz0qspo
  Running command git clone -q https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-_yz0qspo


In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
from torch.nn import Linear
import torch.nn.functional as F
from tqdm import tqdm
from torch_geometric.loader import NeighborSampler
from torch_geometric.nn import SAGEConv, GCNConv, GATConv, LEConv, GATv2Conv
import os.path as osp
import pandas as pd
import numpy as np
import collections
from pandas.core.common import flatten

### Dataset
<b>Dataset</b>: We use [obgn-products](https://ogb.stanford.edu/docs/nodeprop/#ogbn-products) dataset which is an undirected and unweighted graph, representing an Amazon product co-purchasing network. Nodes represent products sold in Amazon, and edges between two products indicate that the products are purchased together. Node features represent bag-of-words features taken from the product descriptions. The goal is to predict the category of a product in a multi-class classification setup, where the 47 top-level categories are used for target labels making it a <b>Node Classification Task</b>. 

In [4]:
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
from pandas.core.common import flatten
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(16.7,8.27)})
sns.set_theme(style="ticks")
import collections
from scipy.special import softmax

In [5]:
fp = '/content/drive/MyDrive/final_project/'
dataset = PygNodePropPredDataset('ogbn-products', fp)

In [6]:
# split_idx contains a dictionary of train, validation and test node indices
split_idx = dataset.get_idx_split()
# predefined ogb evaluator method used for validation of predictions
evaluator = Evaluator(name='ogbn-products')

### EDA

In [7]:
print('Number of training nodes:', split_idx['train'].size(0))
print('Number of validation nodes:', split_idx['valid'].size(0))
print('Number of test nodes:', split_idx['test'].size(0))

Number of training nodes: 196615
Number of validation nodes: 39323
Number of test nodes: 2213091


In [8]:
# loading the dataset
data = dataset[0]

In [9]:
# basic graph statistics of ogb-product graph
print("Number of nodes in the graph:", data.num_nodes)
print("Number of edges in the graph:", data.num_edges)
print("Node feature matrix with shape:", data.x.shape) # [num_nodes, num_node_features]
print("Graph connectivity in COO format with shape:", data.edge_index.shape) # [2, num_edges]
print("Target to train against :", data.y.shape) 
print("Node feature length", dataset.num_features)

Number of nodes in the graph: 2449029
Number of edges in the graph: 123718280
Node feature matrix with shape: torch.Size([2449029, 100])
Graph connectivity in COO format with shape: torch.Size([2, 123718280])
Target to train against : torch.Size([2449029, 1])
Node feature length 100


In [10]:
# there are indeed 47 unique categories of product, as expected
data.y.unique()

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46])

### train_loader
`train_loader` computes the k-hop neighborhood of a batch of nodes, and returns, for each layer, a bipartite graph object, holding the bipartite edges `edge_index`, the index `e_id` of the original edges, and the size/shape `size` of the bipartite graph.

Here we are sampling 20 one-hop neighbours, 15 two-hop neighbours and 10 three-hop neighbours.

In [11]:
train_idx = split_idx['train']
train_loader = NeighborSampler(data.edge_index, node_idx=train_idx,
                               sizes=[20, 15, 10], batch_size=256,
                               shuffle=True)

### Loading

In [12]:
# load integer to real product category from label mapping provided inside the dataset
df = pd.read_csv('/content/drive/MyDrive/final_project/ogbn_products/mapping/labelidx2productcategory.csv.gz')

In [13]:
# creating a dictionary of product category and corresponding integer label
label_idx, prod_cat = df.iloc[: ,0].values, df.iloc[: ,1].values
label_mapping = dict(zip(label_idx, prod_cat))
print("Label mapping", label_mapping)

Label mapping {0: 'Home & Kitchen', 1: 'Health & Personal Care', 2: 'Beauty', 3: 'Sports & Outdoors', 4: 'Books', 5: 'Patio, Lawn & Garden', 6: 'Toys & Games', 7: 'CDs & Vinyl', 8: 'Cell Phones & Accessories', 9: 'Grocery & Gourmet Food', 10: 'Arts, Crafts & Sewing', 11: 'Clothing, Shoes & Jewelry', 12: 'Electronics', 13: 'Movies & TV', 14: 'Software', 15: 'Video Games', 16: 'Automotive', 17: 'Pet Supplies', 18: 'Office Products', 19: 'Industrial & Scientific', 20: 'Musical Instruments', 21: 'Tools & Home Improvement', 22: 'Magazine Subscriptions', 23: 'Baby Products', 24: nan, 25: 'Appliances', 26: 'Kitchen & Dining', 27: 'Collectibles & Fine Art', 28: 'All Beauty', 29: 'Luxury Beauty', 30: 'Amazon Fashion', 31: 'Computers', 32: 'All Electronics', 33: 'Purchase Circles', 34: 'MP3 Players & Accessories', 35: 'Gift Cards', 36: 'Office & School Supplies', 37: 'Home Improvement', 38: 'Camera & Photo', 39: 'GPS & Navigation', 40: 'Digital Music', 41: 'Car Electronics', 42: 'Baby', 43: 'Kin

In [14]:
y = data.y.tolist()
y = list(flatten(y))
count_y = collections.Counter(y)
print(count_y)

Counter({4: 668950, 7: 172199, 6: 158771, 3: 151061, 12: 131886, 2: 116043, 0: 114294, 8: 110796, 1: 109832, 13: 101541, 16: 83594, 21: 80795, 9: 67358, 10: 52345, 18: 49019, 24: 45406, 17: 42337, 5: 40715, 11: 32937, 42: 32500, 15: 26911, 20: 22575, 19: 17438, 23: 3653, 14: 3079, 25: 3024, 28: 1969, 29: 1561, 43: 1399, 22: 879, 36: 630, 44: 566, 26: 553, 37: 514, 32: 513, 31: 418, 30: 277, 27: 259, 34: 154, 38: 91, 41: 61, 35: 44, 39: 37, 33: 29, 45: 9, 40: 6, 46: 1})


In [15]:
x = data.x.to(device)
y = data.y.squeeze().to(device)

### Models

In [16]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers):
        super(SAGE, self).__init__()

        self.num_layers = num_layers

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adjs):
        for i, (edge_index, _, size) in enumerate(adjs):
            xs = []
            x_target = x[:size[1]]
            x = self.convs[i]((x, x_target), edge_index)
            if i != self.num_layers - 1:
                x = F.relu(x)
                x = F.dropout(x, p=0.25, training=self.training)
            xs.append(x)
            if i == 0: 
                x_all = torch.cat(xs, dim=0)
                layer_1_embeddings = x_all
            elif i == 1:
                x_all = torch.cat(xs, dim=0)
                layer_2_embeddings = x_all
            elif i == 2:
                x_all = torch.cat(xs, dim=0)
                layer_3_embeddings = x_all    
        return layer_3_embeddings.log_softmax(dim=-1)

    def inference(self, x_all):
        pbar = tqdm(total=x_all.size(0) * self.num_layers)
        pbar.set_description('Evaluating')
        total_edges = 0
        for i in range(self.num_layers):
            xs = []
            for batch_size, n_id, adj in test_loader:
                edge_index, _, size = adj.to(device)
                total_edges += edge_index.size(1)
                x = x_all[n_id].to(device)
                x_target = x[:size[1]]
                x = self.convs[i]((x, x_target), edge_index)
                if i != self.num_layers - 1:
                    x = F.relu(x)
                xs.append(x)
                pbar.update(batch_size)

            if i == 0: 
                x_all = torch.cat(xs, dim=0)
                layer_1_embeddings = x_all
            elif i == 1:
                x_all = torch.cat(xs, dim=0)
                layer_2_embeddings = x_all
            elif i == 2:
                x_all = torch.cat(xs, dim=0)
                layer_3_embeddings = x_all
        pbar.close()
        return layer_3_embeddings.log_softmax(dim=-1)

In [17]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers):
        super(GCN, self).__init__()

        self.num_layers = num_layers

        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
        self.convs.append(GCNConv(hidden_channels, out_channels))

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adjs):
        for i, (edge_index, _, size) in enumerate(adjs):
            xs = []
            x_target = x[:size[1]]
            x = self.convs[i]((x, x_target), edge_index)
            if i != self.num_layers - 1:
                x = F.relu(x)
                x = F.dropout(x, p=0.25, training=self.training)
            xs.append(x)
            if i == 0: 
                x_all = torch.cat(xs, dim=0)
                layer_1_embeddings = x_all
            elif i == 1:
                x_all = torch.cat(xs, dim=0)
                layer_2_embeddings = x_all
            elif i == 2:
                x_all = torch.cat(xs, dim=0)
                layer_3_embeddings = x_all    
        return layer_3_embeddings.log_softmax(dim=-1)

    def inference(self, x_all):
        pbar = tqdm(total=x_all.size(0) * self.num_layers)
        pbar.set_description('Evaluating')
        total_edges = 0
        for i in range(self.num_layers):
            xs = []
            for batch_size, n_id, adj in test_loader:
                edge_index, _, size = adj.to(device)
                total_edges += edge_index.size(1)
                x = x_all[n_id].to(device)
                x_target = x[:size[1]]
                x = self.convs[i]((x, x_target), edge_index)
                if i != self.num_layers - 1:
                    x = F.relu(x)
                xs.append(x)
                pbar.update(batch_size)

            if i == 0: 
                x_all = torch.cat(xs, dim=0)
                layer_1_embeddings = x_all
            elif i == 1:
                x_all = torch.cat(xs, dim=0)
                layer_2_embeddings = x_all
            elif i == 2:
                x_all = torch.cat(xs, dim=0)
                layer_3_embeddings = x_all
        pbar.close()
        return layer_3_embeddings.log_softmax(dim=-1)

In [18]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 heads):
        super(GAT, self).__init__()

        self.num_layers = num_layers

        self.convs = torch.nn.ModuleList()
        self.convs.append(GATConv(dataset.num_features, hidden_channels,
                                  heads))
        for _ in range(num_layers - 2):
            self.convs.append(
                GATConv(heads * hidden_channels, hidden_channels, heads))
        self.convs.append(
            GATConv(heads * hidden_channels, out_channels, heads, concat=False))

        self.skips = torch.nn.ModuleList()
        self.skips.append(Linear(dataset.num_features, hidden_channels * heads))
        for _ in range(num_layers - 2):
            self.skips.append(
                Linear(hidden_channels * heads, hidden_channels * heads))
        self.skips.append(Linear(hidden_channels * heads, out_channels))

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for skip in self.skips:
            skip.reset_parameters()

    def forward(self, x, adjs):
        for i, (edge_index, _, size) in enumerate(adjs):
            x_target = x[:size[1]]
            x = self.convs[i]((x, x_target), edge_index)
            if i != self.num_layers - 1:
                x = F.elu(x)
                x = F.dropout(x, p=0.25, training=self.training)

            x = x + self.skips[i](x_target)
        return x.log_softmax(dim=-1)

    def inference(self, x_all):
        pbar = tqdm(total=x_all.size(0) * self.num_layers)
        pbar.set_description('Evaluating')
        total_edges = 0
        for i in range(self.num_layers):
            xs = []
            for batch_size, n_id, adj in test_loader:
                edge_index, _, size = adj.to(device)
                total_edges += edge_index.size(1)
                x = x_all[n_id].to(device)
                x_target = x[:size[1]]
                x = self.convs[i]((x, x_target), edge_index)
                if i != self.num_layers - 1:
                    x = F.elu(x)
                
                x = x + self.skips[i](x_target)
                xs.append(x.cpu())
                pbar.update(batch_size)
            x_all = torch.cat(xs, dim=0)

        pbar.close()
        return x_all

In [19]:
class GATv2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 heads):
        super(GATv2, self).__init__()
        self.num_layers = num_layers
        self.convs = torch.nn.ModuleList()
        self.convs.append(GATv2Conv(dataset.num_features, hidden_channels,
                                  heads))
        for _ in range(num_layers - 2):
            self.convs.append(
                GATv2Conv(heads * hidden_channels, hidden_channels, heads))
        self.convs.append(
            GATv2Conv(heads * hidden_channels, out_channels, heads, concat=False))

        self.skips = torch.nn.ModuleList()
        self.skips.append(Linear(dataset.num_features, hidden_channels * heads))
        for _ in range(num_layers - 2):
            self.skips.append(
                Linear(hidden_channels * heads, hidden_channels * heads))
        self.skips.append(Linear(hidden_channels * heads, out_channels))

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for skip in self.skips:
            skip.reset_parameters()

    def forward(self, x, adjs):
        for i, (edge_index, _, size) in enumerate(adjs):
            x_target = x[:size[1]]
            x = self.convs[i]((x, x_target), edge_index)
            if i != self.num_layers - 1:
                x = F.elu(x)
                x = F.dropout(x, p=0.25, training=self.training)

            x = x + self.skips[i](x_target)
        return x.log_softmax(dim=-1)

    def inference(self, x_all):
        pbar = tqdm(total=x_all.size(0) * self.num_layers)
        pbar.set_description('Evaluating')
        total_edges = 0
        for i in range(self.num_layers):
            xs = []
            for batch_size, n_id, adj in test_loader:
                edge_index, _, size = adj.to(device)
                total_edges += edge_index.size(1)
                x = x_all[n_id].to(device)
                x_target = x[:size[1]]
                x = self.convs[i]((x, x_target), edge_index)
                if i != self.num_layers - 1:
                    x = F.elu(x)
                
                x = x + self.skips[i](x_target)
                xs.append(x.cpu())
                pbar.update(batch_size)
            x_all = torch.cat(xs, dim=0)

        pbar.close()
        return x_all

In [20]:
class LeGCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers):
        super(LeGCN, self).__init__()

        self.num_layers = num_layers

        self.convs = torch.nn.ModuleList()
        self.convs.append(LEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(LEConv(hidden_channels, hidden_channels))
        self.convs.append(LEConv(hidden_channels, out_channels))

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adjs):
        for i, (edge_index, _, size) in enumerate(adjs):
            xs = []
            x_target = x[:size[1]]
            x = self.convs[i]((x, x_target), edge_index)
            if i != self.num_layers - 1:
                x = F.relu(x)
                x = F.dropout(x, p=0.25, training=self.training)
            xs.append(x)
            if i == 0: 
                x_all = torch.cat(xs, dim=0)
                layer_1_embeddings = x_all
            elif i == 1:
                x_all = torch.cat(xs, dim=0)
                layer_2_embeddings = x_all
            elif i == 2:
                x_all = torch.cat(xs, dim=0)
                layer_3_embeddings = x_all    
        return layer_3_embeddings.log_softmax(dim=-1)

    def inference(self, x_all):
        pbar = tqdm(total=x_all.size(0) * self.num_layers)
        pbar.set_description('Evaluating')
        total_edges = 0
        for i in range(self.num_layers):
            xs = []
            for batch_size, n_id, adj in test_loader:
                edge_index, _, size = adj.to(device)
                total_edges += edge_index.size(1)
                x = x_all[n_id].to(device)
                x_target = x[:size[1]]
                x = self.convs[i]((x, x_target), edge_index)
                if i != self.num_layers - 1:
                    x = F.relu(x)
                xs.append(x)
                pbar.update(batch_size)
            if i == 0: 
                x_all = torch.cat(xs, dim=0)
                layer_1_embeddings = x_all
            elif i == 1:
                x_all = torch.cat(xs, dim=0)
                layer_2_embeddings = x_all
            elif i == 2:
                x_all = torch.cat(xs, dim=0)
                layer_3_embeddings = x_all
        pbar.close()
        return layer_3_embeddings.log_softmax(dim=-1)

### Training loop

While using test_loader for evaluation, we should delete the train_loader. Otherwise, Google Colab is crashing due to memory limitation.

In [21]:
def train(epoch):
    model.train()

    pbar = tqdm(total=train_idx.size(0))
    pbar.set_description(f'Epoch {epoch:02d}')

    total_loss = total_correct = 0
    for batch_size, n_id, adjs in train_loader:
        adjs = [adj.to(device) for adj in adjs]

        optimizer.zero_grad()
        out = model(x[n_id], adjs)
        loss = F.nll_loss(out, y[n_id[:batch_size]])
        loss.backward()
        optimizer.step()

        total_loss += float(loss)
        total_correct += int(out.argmax(dim=-1).eq(y[n_id[:batch_size]]).sum())
        pbar.update(batch_size)

    pbar.close()

    loss = total_loss / len(train_loader)
    approx_acc = total_correct / train_idx.size(0)
    return loss, approx_acc

In [22]:
# model = SAGE(in_channels=dataset.num_features, hidden_channels=256,
            #  out_channels=dataset.num_classes, num_layers=3)

# model = GCN(in_channels=dataset.num_features, hidden_channels=256,
#             out_channels=dataset.num_classes, num_layers=3)

# model = LeGCN(in_channels=dataset.num_features, hidden_channels=256, 
#               out_channels=dataset.num_classes, num_layers=3)

# model = GAT(in_channels=dataset.num_features, hidden_channels=128,
#               out_channels=dataset.num_classes, num_layers=4, heads=4)

model = GATv2(in_channels=dataset.num_features, hidden_channels=128,
              out_channels=dataset.num_classes, num_layers=4, heads=4)

model = model.to(device)

In [23]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.004)

for epoch in range(1, 21):
    loss, acc = train(epoch)
    print(f'Epoch {epoch:02d}, Loss: {loss:.4f}, Approx. Train: {acc:.4f}')

Epoch 01: 100%|██████████| 196615/196615 [05:04<00:00, 645.49it/s]


Epoch 01, Loss: 0.4890, Approx. Train: 0.8714


Epoch 02: 100%|██████████| 196615/196615 [05:03<00:00, 647.49it/s]


Epoch 02, Loss: 0.3656, Approx. Train: 0.9012


Epoch 03: 100%|██████████| 196615/196615 [05:04<00:00, 645.90it/s]


...

Epoch 18, Loss: 0.3016, Approx. Train: 0.9179


Epoch 19: 100%|██████████| 196615/196615 [05:29<00:00, 596.80it/s]


Epoch 19, Loss: 0.3013, Approx. Train: 0.9175


Epoch 20: 100%|██████████| 196615/196615 [05:20<00:00, 613.74it/s]

Epoch 20, Loss: 0.2998, Approx. Train: 0.9178


The loss function: negative log-likelihood.

Screenshot 2022-04-06 at 13.51.26.png

### Saving the model


In [24]:
# saving model in Google Drive
MODEL_PATH = '/content/drive/MyDrive/final_project/models/model_GATv2.pt'
torch.save(model, MODEL_PATH)
print(f'Model Saved at {MODEL_PATH}')

Model Saved at /content/drive/MyDrive/final_project/models/model_GATv2.pt


In [ ]:
torch.cuda.empty_cache()

## Testing the model
Computing the accuracies on the validation and test sets.

In [25]:
test_loader = NeighborSampler(data.edge_index, node_idx=None,
                              sizes=[-1], # all neighbours of a node
                              batch_size=256, shuffle=False)

In [26]:
# loading the saved model
model = torch.load(MODEL_PATH, map_location=torch.device(device))
model = model.to(device)

In [27]:
@torch.no_grad()
def compute_accuracy():
    model.eval()
    out = model.inference(x)
    y_true = y.cpu().unsqueeze(-1)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': y_true[split_idx["train"]],
        'y_pred': y_pred[split_idx["train"]],
    })['acc']

    valid_acc = evaluator.eval({
        'y_true': y_true[split_idx["valid"]],
        'y_pred': y_pred[split_idx["valid"]],
    })['acc']

    test_acc = evaluator.eval({
        'y_true': y_true[split_idx["test"]],
        'y_pred': y_pred[split_idx["test"]],
    })['acc']

    return train_acc, valid_acc, test_acc

In [29]:
train_acc, valid_acc, test_acc = compute_accuracy()

Evaluating: 100%|██████████| 7347087/7347087 [01:58<00:00, 61869.61it/s]


In [31]:
print('Train accuracy:', np.round(train_acc*100, 2))
print('Validation accuracy:', np.round(valid_acc*100, 2))
print('Test accuracy:', np.round(test_acc*100, 2))

Train accuracy: 93.38
Validation accuracy: 91.57
Test accuracy: 84.27


# References



1. [Inductive Representation Learning on Large Graphs](https://arxiv.org/pdf/1706.02216.pdf)

2. http://web.stanford.edu/class/cs224w/slides/17-scalable.pdf

3. [A Voyage through Graph Machine Learning Universe: Motivation, Applications, Datasets, Graph ML Libraries, Graph Databases](https://sachinsharma9780.medium.com/)

3. https://medium.com/pinterest-engineering/pinsage-a-new-graph-convolutional-neural-network-for-web-scale-recommender-systems-88795a107f48

4. https://eng.uber.com/uber-eats-graph-learning/


